In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Layer
from tensorflow.python.framework.ops import disable_eager_execution

In [ ]:
print(tf.__version__)

2.9.2


In [ ]:
non_eager_execution = False
if non_eager_execution: 
   tf.compat.v1.disable_eager_execution()
else:
   tf.config.run_functions_eagerly(True)

print("Eager execution is: " + str(tf.executing_eagerly()))

Eager execution is: True




*(1) define the custom RNN layer*

In [ ]:
class RNN_simple_layer(Layer):

    def __init__(self, neurons_numbers, return_sequences=False, **kwargs):
        super(RNN_simple_layer, self).__init__()
        self.neurons_numbers = neurons_numbers
        self.return_sequences = return_sequences

    def build(self, input_shape):
        self.kernel_input = self.add_weight('kernel_input', shape=[int(input_shape[-1]), self.neurons_numbers],
                                            initializer="random_normal", trainable=True)
        self.kernel_state = self.add_weight('kernel_state', shape=[self.neurons_numbers, self.neurons_numbers],
                                            initializer="random_normal", trainable=True)
        self.bias = self.add_weight('bias', shape=[self.neurons_numbers, ], initializer="zeros", trainable=True)

    @tf.function
    def call(self, inputs):
        sequence_length = inputs.shape[-2]

        # outputs = []
        outputs = tf.TensorArray(tf.float32, size=sequence_length, dynamic_size=True)

        for i_step in range(sequence_length):
            if i_step == 0:
                states = tf.zeros(shape=[tf.shape(inputs)[0], self.neurons_numbers])

            output_i_step = tf.tanh(
                tf.matmul(inputs[:, i_step, :], self.kernel_input) + tf.matmul(states, self.kernel_state) + self.bias)
            states = output_i_step
            outputs = outputs.write(i_step, output_i_step)

        # output the RNN resuts
        if self.return_sequences == True:
            output = tf.transpose(outputs.stack(), [1, 0, 2])
        else:
            output = outputs.stack()[-1]
        return output

    def get_config(self):
        config = super(RNN_simple_layer, self).get_config()
        config.update({
            'neurons_numbers': self.neurons_numbers,
            'return_sequences': self.return_sequences
        })
        return config


*(2) Test the customized simple RNN layer*

In [ ]:
# test with customized layers
RNN_layer    = RNN_simple_layer(4,return_sequences=True)
# RNN_layer    = RNN_simple_layer(4,return_sequences=False)
input_array  = tf.ones((32,10,8))
output_array = RNN_layer(input_array)

print('RNN input tensor shape:' + str(input_array.shape))
print('RNN input tensor has batch size:' + str(input_array.shape[0]))
print('RNN input tensor has time samples:' + str(input_array.shape[-2]))
print('RNN input tensor has features of each sample:' + str(input_array.shape[-1]))
print('RNN output tensor shape:' + str(output_array.shape))

RNN input tensor shape:(32, 10, 8)
RNN input tensor has batch size:32
RNN input tensor has time samples:10
RNN input tensor has features of each sample:8
RNN output tensor shape:(32, 10, 4)


*(3) Build a dummy Tensorflow model*

In [ ]:
inputs  = tf.keras.Input(shape=(10,20))
x       = tf.keras.layers.Conv1D(10,3)(inputs)
x       = RNN_simple_layer(4,return_sequences=True)(x)
outputs = tf.keras.layers.Dense(30, activation="relu")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="test_model")
model.summary()


Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 20)]          0         
                                                                 
 conv1d (Conv1D)             (None, 8, 10)             610       
                                                                 
 rnn_simple_layer_1 (RNN_sim  (None, None, 4)          60        
 ple_layer)                                                      
                                                                 
 dense (Dense)               (None, None, 30)          150       
                                                                 
Total params: 820
Trainable params: 820
Non-trainable params: 0
_________________________________________________________________


*(4) save the dummy model*

In [ ]:
model.save('Model_simple_RNN')

model_load = tf.keras.models.load_model('Model_simple_RNN')
model_load.summary()

Model: "test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 20)]          0         
                                                                 
 conv1d (Conv1D)             (None, 8, 10)             610       
                                                                 
 rnn_simple_layer_1 (RNN_sim  (None, None, 4)          60        
 ple_layer)                                                      
                                                                 
 dense (Dense)               (None, None, 30)          150       
                                                                 
Total params: 820
Trainable params: 820
Non-trainable params: 0
_________________________________________________________________


*(5) Convert the dummy model to tensorflow-lite model*

In [ ]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_saved_model('Model_simple_RNN')

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
open('Model_simple_RNN_TF28.tflite', 'wb').write(tfmodel)

32184

In [ ]:
# converter               = tf.lite.TFLiteConverter.from_saved_model('Model_simple_RNN')
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_types = [tf.float32]

# tflite_model_file = 'Model_simple_RNN_TF28.tflite'
# tflite_model      = converter.convert()

# with open(tflite_model_file, "wb") as f:
#     f.write(tflite_model)